In [1]:
import pandas as pd
import numpy as np
import duckdb
from datetime import datetime, timedelta

In [2]:
# Criação de um DataFrame de exemplo
data = {
    'id': range(1, 11),
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Alice', 'Grace', 'Hannah', 'Ivy', 'Jack'],
    'age': np.random.randint(20, 50, size=10),
    'signup_date': [datetime(2023, 1, 1) + timedelta(days=i) for i in range(10)],
    'purchase_amount': np.random.uniform(50.5, 500.75, size=10)
}
df = pd.DataFrame(data)

In [3]:
# DuckDB em memória
con = duckdb.connect(database=':memory:')

In [4]:
# DataFrame como uma tabela DuckDB
con.register('df', df)

In [5]:
# Executar consulta 
result_1 = con.sql('FROM df')
print("\n1. Todos os dados da tabela df:")
print(result_1)


1. Todos os dados da tabela df:
┌───────┬─────────┬───────┬─────────────────────┬────────────────────┐
│  id   │  name   │  age  │     signup_date     │  purchase_amount   │
│ int64 │ varchar │ int64 │    timestamp_ns     │       double       │
├───────┼─────────┼───────┼─────────────────────┼────────────────────┤
│     1 │ Alice   │    32 │ 2023-01-01 00:00:00 │ 219.35184330198967 │
│     2 │ Bob     │    28 │ 2023-01-02 00:00:00 │ 107.54957526210248 │
│     3 │ Charlie │    43 │ 2023-01-03 00:00:00 │  74.26470488544021 │
│     4 │ David   │    47 │ 2023-01-04 00:00:00 │  306.3556057851411 │
│     5 │ Eve     │    30 │ 2023-01-05 00:00:00 │  169.2540914937183 │
│     6 │ Alice   │    38 │ 2023-01-06 00:00:00 │   59.9378069382538 │
│     7 │ Grace   │    37 │ 2023-01-07 00:00:00 │ 124.33150352875784 │
│     8 │ Hannah  │    48 │ 2023-01-08 00:00:00 │ 481.17723532470717 │
│     9 │ Ivy     │    38 │ 2023-01-09 00:00:00 │  304.8140112760028 │
│    10 │ Jack    │    20 │ 2023-01-10 00:00

### Agregação

In [6]:
result_2 = con.sql('SELECT name, COUNT(name) as count FROM df GROUP BY name')
print("\n2. Contagem de IDs por nome:")
print(result_2)


2. Contagem de IDs por nome:
┌─────────┬───────┐
│  name   │ count │
│ varchar │ int64 │
├─────────┼───────┤
│ Alice   │     2 │
│ Charlie │     1 │
│ Bob     │     1 │
│ Hannah  │     1 │
│ Ivy     │     1 │
│ David   │     1 │
│ Jack    │     1 │
│ Eve     │     1 │
│ Grace   │     1 │
└─────────┴───────┘



### Função de Janela

In [7]:
result_3 = con.sql('SELECT name, age, row_number() \
            OVER (ORDER BY age) as row_num FROM df')
print("\n3. Função de janela com row_number ordenado por idade:")
print(result_3)


3. Função de janela com row_number ordenado por idade:
┌─────────┬───────┬─────────┐
│  name   │  age  │ row_num │
│ varchar │ int64 │  int64  │
├─────────┼───────┼─────────┤
│ Jack    │    20 │       1 │
│ Bob     │    28 │       2 │
│ Eve     │    30 │       3 │
│ Alice   │    32 │       4 │
│ Grace   │    37 │       5 │
│ Alice   │    38 │       6 │
│ Ivy     │    38 │       7 │
│ Charlie │    43 │       8 │
│ David   │    47 │       9 │
│ Hannah  │    48 │      10 │
├─────────┴───────┴─────────┤
│ 10 rows         3 columns │
└───────────────────────────┘



### Fuções Matemáticas

In [8]:
# Funções matemática
result_4 = con.sql('SELECT name, LOG(purchase_amount) as log_purchase FROM df')
print("\n4. Logaritmo da quantidade comprada (purchase_amount):")
print(result_4)


4. Logaritmo da quantidade comprada (purchase_amount):
┌─────────┬────────────────────┐
│  name   │    log_purchase    │
│ varchar │       double       │
├─────────┼────────────────────┤
│ Alice   │ 2.3411412883079525 │
│ Bob     │  2.031608699598522 │
│ Charlie │ 1.8707824595825535 │
│ David   │ 2.4862258315890733 │
│ Eve     │ 2.2285391759785536 │
│ Alice   │ 1.7777008485188863 │
│ Grace   │  2.094581185561858 │
│ Hannah  │  2.682305072519164 │
│ Ivy     │  2.484034926184611 │
│ Jack    │  2.110522779049739 │
├─────────┴────────────────────┤
│ 10 rows            2 columns │
└──────────────────────────────┘



### Subqueries na cláusula FROM

In [9]:
result_6 = con.sql('SELECT name, avg_purchase_amount \
            FROM (SELECT name, AVG(purchase_amount) as avg_purchase_amount \
                  FROM df \
                  GROUP BY name) \
            WHERE avg_purchase_amount > 250')
print("\n6. Subquery com filtragem de média de compras maior que 250:")
print(result_6)


6. Subquery com filtragem de média de compras maior que 250:
┌─────────┬─────────────────────┐
│  name   │ avg_purchase_amount │
│ varchar │       double        │
├─────────┼─────────────────────┤
│ David   │   306.3556057851411 │
│ Hannah  │  481.17723532470717 │
│ Ivy     │   304.8140112760028 │
└─────────┴─────────────────────┘



### Manipulação de strings 

In [11]:
result_7 = con.sql('SELECT name, UPPER(name) as upper_name FROM df')
print("\n7. Nome em letras maiúsculas (UPPER):")
print(result_7)


7. Nome em letras maiúsculas (UPPER):
┌─────────┬────────────┐
│  name   │ upper_name │
│ varchar │  varchar   │
├─────────┼────────────┤
│ Alice   │ ALICE      │
│ Bob     │ BOB        │
│ Charlie │ CHARLIE    │
│ David   │ DAVID      │
│ Eve     │ EVE        │
│ Alice   │ ALICE      │
│ Grace   │ GRACE      │
│ Hannah  │ HANNAH     │
│ Ivy     │ IVY        │
│ Jack    │ JACK       │
├─────────┴────────────┤
│ 10 rows    2 columns │
└──────────────────────┘



### Consultas SQL diretamente em arquivos sem carregá-los

In [12]:
result_8A = con.sql("SELECT * FROM 'data/data.csv'")
print("\n8A. Consulta direta em CSV:")
print(result_8A)


8A. Consulta direta em CSV:
┌─────────┬────────────────┬───────┬─────────────────┬───────────────┐
│ user_id │      name      │  age  │ purchase_amount │ purchase_date │
│  int64  │    varchar     │ int64 │     double      │     date      │
├─────────┼────────────────┼───────┼─────────────────┼───────────────┤
│     101 │ John Doe       │    28 │          250.75 │ 2024-08-01    │
│     102 │ Jane Smith     │    34 │           300.0 │ 2024-08-02    │
│     103 │ Alice Johnson  │    23 │           120.5 │ 2024-08-03    │
│     104 │ Chris Evans    │    45 │           450.0 │ 2024-08-04    │
│     105 │ Emily Davis    │    30 │            80.0 │ 2024-08-05    │
│     106 │ Michael Brown  │    40 │          199.99 │ 2024-08-06    │
│     107 │ Jessica Taylor │    29 │          349.99 │ 2024-08-07    │
│     108 │ Daniel Wilson  │    35 │           220.0 │ 2024-08-08    │
│     109 │ Laura Clark    │    27 │          110.75 │ 2024-08-09    │
│     110 │ Robert Lewis   │    31 │           5

In [13]:
result_8B = con.sql("SELECT * FROM 'data/data.parquet'")
print("\n8B. Consulta direta em parquet:")
print(result_8B)


8B. Consulta direta em parquet:
┌─────────┬────────────────┬───────┬─────────────────┬───────────────┐
│ user_id │      name      │  age  │ purchase_amount │ purchase_date │
│  int64  │    varchar     │ int64 │     double      │    varchar    │
├─────────┼────────────────┼───────┼─────────────────┼───────────────┤
│     101 │ John Doe       │    28 │          250.75 │ 2024-08-01    │
│     102 │ Jane Smith     │    34 │           300.0 │ 2024-08-02    │
│     103 │ Alice Johnson  │    23 │           120.5 │ 2024-08-03    │
│     104 │ Chris Evans    │    45 │           450.0 │ 2024-08-04    │
│     105 │ Emily Davis    │    30 │            80.0 │ 2024-08-05    │
│     106 │ Michael Brown  │    40 │          199.99 │ 2024-08-06    │
│     107 │ Jessica Taylor │    29 │          349.99 │ 2024-08-07    │
│     108 │ Daniel Wilson  │    35 │           220.0 │ 2024-08-08    │
│     109 │ Laura Clark    │    27 │          110.75 │ 2024-08-09    │
│     110 │ Robert Lewis   │    31 │        

In [14]:
result_8C = con.sql("SELECT * FROM 'data/data.json'")
print("\n8C. Consulta direta em Json:")
print(result_8C)


8C. Consulta direta em Json:
┌─────────┬────────────────┬───────┬─────────────────┬───────────────┐
│ user_id │      name      │  age  │ purchase_amount │ purchase_date │
│  int64  │    varchar     │ int64 │     double      │     date      │
├─────────┼────────────────┼───────┼─────────────────┼───────────────┤
│     101 │ John Doe       │    28 │          250.75 │ 2024-08-01    │
│     102 │ Jane Smith     │    34 │           300.0 │ 2024-08-02    │
│     103 │ Alice Johnson  │    23 │           120.5 │ 2024-08-03    │
│     104 │ Chris Evans    │    45 │           450.0 │ 2024-08-04    │
│     105 │ Emily Davis    │    30 │            80.0 │ 2024-08-05    │
│     106 │ Michael Brown  │    40 │          199.99 │ 2024-08-06    │
│     107 │ Jessica Taylor │    29 │          349.99 │ 2024-08-07    │
│     108 │ Daniel Wilson  │    35 │           220.0 │ 2024-08-08    │
│     109 │ Laura Clark    │    27 │          110.75 │ 2024-08-09    │
│     110 │ Robert Lewis   │    31 │           

In [15]:
result_8D = con.sql("SELECT LastName  FROM \
        sqlite_scan('data/northwind.db', 'employees') limit 10")
print("\n8D. Consulta direta em SQLite:")
print(result_8D)


8D. Consulta direta em SQLite:
┌───────────┐
│ LastName  │
│  varchar  │
├───────────┤
│ Davolio   │
│ Fuller    │
│ Leverling │
│ Peacock   │
│ Buchanan  │
│ Suyama    │
│ King      │
│ Callahan  │
│ Dodsworth │
└───────────┘



### Datas e horas

In [16]:
result_9 = con.sql("SELECT signup_date, YEAR(signup_date) \
            as year, MONTH(signup_date) as month,\
            DAY(signup_date) as day FROM df")
print("\n9. Manipulação de datas (YEAR, MONTH, DAY):")
print(result_9)


9. Manipulação de datas (YEAR, MONTH, DAY):
┌─────────────────────┬───────┬───────┬───────┐
│     signup_date     │ year  │ month │  day  │
│    timestamp_ns     │ int64 │ int64 │ int64 │
├─────────────────────┼───────┼───────┼───────┤
│ 2023-01-01 00:00:00 │  2023 │     1 │     1 │
│ 2023-01-02 00:00:00 │  2023 │     1 │     2 │
│ 2023-01-03 00:00:00 │  2023 │     1 │     3 │
│ 2023-01-04 00:00:00 │  2023 │     1 │     4 │
│ 2023-01-05 00:00:00 │  2023 │     1 │     5 │
│ 2023-01-06 00:00:00 │  2023 │     1 │     6 │
│ 2023-01-07 00:00:00 │  2023 │     1 │     7 │
│ 2023-01-08 00:00:00 │  2023 │     1 │     8 │
│ 2023-01-09 00:00:00 │  2023 │     1 │     9 │
│ 2023-01-10 00:00:00 │  2023 │     1 │    10 │
├─────────────────────┴───────┴───────┴───────┤
│ 10 rows                           4 columns │
└─────────────────────────────────────────────┘



### CASE

In [17]:
result_10 = con.sql("SELECT name, \
                            CASE WHEN age > 30  THEN 'Mais velho' \
                            ELSE 'Jovem' \
                            END AS age_group FROM df")
print("\n10. Utilização de CASE para categorizar idade:")
print(result_10)


10. Utilização de CASE para categorizar idade:
┌─────────┬────────────┐
│  name   │ age_group  │
│ varchar │  varchar   │
├─────────┼────────────┤
│ Alice   │ Mais velho │
│ Bob     │ Jovem      │
│ Charlie │ Mais velho │
│ David   │ Mais velho │
│ Eve     │ Jovem      │
│ Alice   │ Mais velho │
│ Grace   │ Mais velho │
│ Hannah  │ Mais velho │
│ Ivy     │ Mais velho │
│ Jack    │ Jovem      │
├─────────┴────────────┤
│ 10 rows    2 columns │
└──────────────────────┘



### Funções JSON 

In [ ]:
df['json_data'] = '{"key1": "value1", "key2": "value2"}'

# atualiza
con.register('df', df)

result_11 = con.sql("SELECT name, json_data->'key1' as key1_value FROM df")

print("\n11. Extração de valor de JSON:")

print(result_11)


11. Extração de valor de JSON:
┌─────────┬────────────┐
│  name   │ key1_value │
│ varchar │    json    │
├─────────┼────────────┤
│ Alice   │ "value1"   │
│ Bob     │ "value1"   │
│ Charlie │ "value1"   │
│ David   │ "value1"   │
│ Eve     │ "value1"   │
│ Alice   │ "value1"   │
│ Grace   │ "value1"   │
│ Hannah  │ "value1"   │
│ Ivy     │ "value1"   │
│ Jack    │ "value1"   │
├─────────┴────────────┤
│ 10 rows    2 columns │
└──────────────────────┘



### Join com subquery

In [ ]:
result_12 = con.sql("""
    SELECT a.name, b.age_group FROM df a 
    JOIN (SELECT name, CASE WHEN age > 30 THEN 'Older' ELSE 'Younger' END AS age_group FROM df) b 
    ON a.name = b.name
    """)

print("\n12. Join com subquery:")

print(result_12)


12. Join com subquery:
┌─────────┬───────────┐
│  name   │ age_group │
│ varchar │  varchar  │
├─────────┼───────────┤
│ Alice   │ Older     │
│ Bob     │ Younger   │
│ Charlie │ Older     │
│ David   │ Older     │
│ Eve     │ Younger   │
│ Alice   │ Older     │
│ Grace   │ Older     │
│ Hannah  │ Older     │
│ Ivy     │ Older     │
│ Jack    │ Younger   │
│ Alice   │ Older     │
│ Alice   │ Older     │
├─────────┴───────────┤
│ 12 rows   2 columns │
└─────────────────────┘



### CTEs (Common Table Expressions)

In [20]:
result_13 = con.sql("""
            WITH AgeGroups AS (
                SELECT name, CASE WHEN age > 30 THEN 'Older' ELSE 'Younger' END AS age_group FROM df
            )
            SELECT * FROM AgeGroups
            """)
print("\n13. Utilização de CTE (Common Table Expressions):")
print(result_13)


13. Utilização de CTE (Common Table Expressions):
┌─────────┬───────────┐
│  name   │ age_group │
│ varchar │  varchar  │
├─────────┼───────────┤
│ Alice   │ Older     │
│ Bob     │ Younger   │
│ Charlie │ Older     │
│ David   │ Older     │
│ Eve     │ Younger   │
│ Alice   │ Older     │
│ Grace   │ Older     │
│ Hannah  │ Older     │
│ Ivy     │ Older     │
│ Jack    │ Younger   │
├─────────┴───────────┤
│ 10 rows   2 columns │
└─────────────────────┘



### Agregação

In [21]:
result_14 = con.sql("SELECT AVG(purchase_amount) as avg_purchase, \
                SUM(purchase_amount) as total_purchase FROM df")
print("\n14. Funções de agregação avançadas (AVG, SUM):")
print(result_14)


14. Funções de agregação avançadas (AVG, SUM):
┌────────────────────┬────────────────────┐
│    avg_purchase    │   total_purchase   │
│       double       │       double       │
├────────────────────┼────────────────────┤
│ 197.60164985062383 │ 1976.0164985062383 │
└────────────────────┴────────────────────┘



### Tabela a partir de uma consulta

In [22]:
con.sql("CREATE TABLE older_customers AS SELECT * FROM df WHERE age > 30")
result_15 = con.sql("SELECT * FROM older_customers")
print("\n15. Tabela criada a partir de uma consulta (older_customers):")
print(result_15)


15. Tabela criada a partir de uma consulta (older_customers):
┌───────┬─────────┬───────┬─────────────────────┬────────────────────┬──────────────────────────────────────┐
│  id   │  name   │  age  │     signup_date     │  purchase_amount   │              json_data               │
│ int64 │ varchar │ int64 │    timestamp_ns     │       double       │               varchar                │
├───────┼─────────┼───────┼─────────────────────┼────────────────────┼──────────────────────────────────────┤
│     1 │ Alice   │    32 │ 2023-01-01 00:00:00 │ 219.35184330198967 │ {"key1": "value1", "key2": "value2"} │
│     3 │ Charlie │    43 │ 2023-01-03 00:00:00 │  74.26470488544021 │ {"key1": "value1", "key2": "value2"} │
│     4 │ David   │    47 │ 2023-01-04 00:00:00 │  306.3556057851411 │ {"key1": "value1", "key2": "value2"} │
│     6 │ Alice   │    38 │ 2023-01-06 00:00:00 │   59.9378069382538 │ {"key1": "value1", "key2": "value2"} │
│     7 │ Grace   │    37 │ 2023-01-07 00:00:00 │ 124.331

### Tabelas temporárias

In [23]:
con.sql("CREATE TEMP TABLE temp_table AS SELECT * FROM df")
result_16 = con.sql("SELECT * FROM temp_table")
print("\n16. Consulta utilizando tabela temporária:")
print(result_16)


16. Consulta utilizando tabela temporária:
┌───────┬─────────┬───────┬─────────────────────┬────────────────────┬──────────────────────────────────────┐
│  id   │  name   │  age  │     signup_date     │  purchase_amount   │              json_data               │
│ int64 │ varchar │ int64 │    timestamp_ns     │       double       │               varchar                │
├───────┼─────────┼───────┼─────────────────────┼────────────────────┼──────────────────────────────────────┤
│     1 │ Alice   │    32 │ 2023-01-01 00:00:00 │ 219.35184330198967 │ {"key1": "value1", "key2": "value2"} │
│     2 │ Bob     │    28 │ 2023-01-02 00:00:00 │ 107.54957526210248 │ {"key1": "value1", "key2": "value2"} │
│     3 │ Charlie │    43 │ 2023-01-03 00:00:00 │  74.26470488544021 │ {"key1": "value1", "key2": "value2"} │
│     4 │ David   │    47 │ 2023-01-04 00:00:00 │  306.3556057851411 │ {"key1": "value1", "key2": "value2"} │
│     5 │ Eve     │    30 │ 2023-01-05 00:00:00 │  169.2540914937183 │ {"key

### Agregação com HAVING

In [24]:
result_18 = con.sql("SELECT name, COUNT(*) as count FROM df GROUP BY name HAVING COUNT(*) > 1")
print("\n18. Agregação com HAVING:")
print(result_18)


18. Agregação com HAVING:
┌─────────┬───────┐
│  name   │ count │
│ varchar │ int64 │
├─────────┼───────┤
│ Alice   │     2 │
└─────────┴───────┘



### Unir tabelas com diferentes colunas usando COALESCE

In [25]:
df2 = pd.DataFrame({
    'id': range(11, 21),
    'name': ['Liam', 'Mia', 'Noah', 'Olivia', 'Sophia', 'William', 'Emma', 'James', 'Ava', 'Benjamin'],
    'age': np.random.randint(20, 50, size=10),
    'purchase_amount': np.random.uniform(50.5, 500.75, size=10)
})

con.register('df2', df2)

result_19 = con.sql("""
SELECT COALESCE(a.name, b.name) AS name, 
       COALESCE(a.age, b.age) AS age,
       COALESCE(a.purchase_amount, b.purchase_amount) AS purchase_amount
FROM df a FULL OUTER JOIN df2 b ON a.id = b.id
""")

print("\n19. Unir duas tabelas usando COALESCE:")

print(result_19)



19. Unir duas tabelas usando COALESCE:
┌──────────┬───────┬────────────────────┐
│   name   │  age  │  purchase_amount   │
│ varchar  │ int64 │       double       │
├──────────┼───────┼────────────────────┤
│ Liam     │    40 │  154.7174694179795 │
│ Mia      │    41 │ 229.53794484782378 │
│ Noah     │    39 │  73.39498314166714 │
│ Olivia   │    20 │ 419.14769942602504 │
│ Sophia   │    39 │ 218.71759450827483 │
│ William  │    28 │  96.43070493026661 │
│ Emma     │    47 │  73.89444001305495 │
│ James    │    33 │  373.2782866815162 │
│ Ava      │    44 │ 238.05034886624824 │
│ Benjamin │    39 │ 395.86713669436483 │
│ Eve      │    30 │  169.2540914937183 │
│ Ivy      │    38 │  304.8140112760028 │
│ Bob      │    28 │ 107.54957526210248 │
│ Jack     │    20 │ 128.98012071012505 │
│ Hannah   │    48 │ 481.17723532470717 │
│ Grace    │    37 │ 124.33150352875784 │
│ Alice    │    32 │ 219.35184330198967 │
│ David    │    47 │  306.3556057851411 │
│ Charlie  │    43 │  74.26470488544